In [ ]:
import numpy as np
import torch
torch.set_default_dtype(torch.float64)
import h5py as h5

import tdg
import tdg.HMC as HMC

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

Let's study a small, hot example for computational simplicity.

In [ ]:
nx = 7
nt = 32
lattice = tdg.Lattice(nx)
spacetime = tdg.Spacetime(nt, lattice)

beta = torch.tensor(1)
mu = torch.tensor(0.125)
h  = torch.tensor([0,0,0], dtype=torch.float64)

contact = (-5)*tdg.LegoSphere([0,0])
V = tdg.Potential(contact)

S = tdg.Action(spacetime, V, beta, mu, h)

We could in principle use one Hamiltonian to do do the HMC Metropolis-Hastings accept/reject step and another to do the molecular dynamics integration.

Here we use the same Hamiltonian for both.

In [ ]:
H = HMC.Hamiltonian(S)
# integrator = HMC.LeapFrog(H, 100, 1)
integrator = HMC.Omelyan(H, 50, 1)
hmc = HMC.MarkovChain(H, integrator)

We decide how many steps we'll do and allocate some space to store the configurations.

In [ ]:
steps = 50   # takes about 1-2 minutes on my laptop
# steps = 1000 # takes about 35 minutes on my laptop

configuration    = spacetime.vector(steps).to(torch.complex128)

Let's start from a configuration sampled from the "quenched" distribution, which ignores the fermion determinant.

In [ ]:
configuration[0] = S.quenched_sample()

Thundercats go!

In [ ]:
for mcmc_step in tqdm(range(1,steps)):
     configuration[mcmc_step] = hmc.step(configuration[mcmc_step-1]).real
        

And now perhaps we wish to visualize some observables, like the action itself

In [ ]:
s = torch.stack([S(c) for c in configuration])

plt.plot(s.real.detach().numpy());

and the total auxiliary field.

In [ ]:
PHI = torch.sum(configuration, (1,2)).detach().numpy()

plt.plot(PHI.real);